In [17]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

df=pd.read_csv("Datos/Transformados/df_limpio.csv", index_col='ID')


In [18]:
df.head()

,Edad,Ingresos,Monto_Inicial,Scoring_Crediticio,Meses_Empleo,Num_Creditos,Ratio_Interes,Ratio_Deuda_Ingresos,Posesion_Hipoteca,Personas_Cargo,...,Estudios_Máster,Tipo_Jornada_Laboral_Desempleado,Tipo_Jornada_Laboral_Jornada completa,Tipo_Jornada_Laboral_Tiempo parcial,Estado_Civil_Divorciado,Estado_Civil_Soltero,Duracion_24,Duracion_36,Duracion_48,Duracion_60
ID,,,,,,,,,,,,,,,,,,,,,
S97R7X,18,61628,83011,397,113,1,8.06,0.45,1,0,...,False,False,False,False,False,False,False,False,True,False
RLGTBY,50,82410,68642,486,14,3,21.96,0.71,1,0,...,False,False,False,True,True,False,False,False,False,False
SKE2P9,37,110810,101859,765,115,3,11.20,0.40,1,1,...,False,False,False,True,True,False,False,False,False,True
E2FB1D,56,16815,157551,643,108,1,21.24,0.72,0,1,...,True,False,False,False,False,True,True,False,False,False
TKSCGH,35,42315,180635,645,114,3,16.95,0.50,1,1,...,False,False,True,False,False,False,False,False,False,False


In [19]:
x = df.drop(columns=['Impago'])
y = df['Impago']

x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.30,
    random_state=6,
    shuffle=True,
    stratify=y)

In [20]:
# PCA

num_cols = x_train.select_dtypes(include=['int64', 'float64']).columns

scaler = MinMaxScaler()
x_train[num_cols] = scaler.fit_transform(x_train[num_cols])
x_test[num_cols] = scaler.transform(x_test[num_cols])

pca = PCA()
X_train_pca = pca.fit_transform(x_train)

explained_variance = np.cumsum(pca.explained_variance_ratio_)
for i, v in enumerate(explained_variance):
    print(f"Componentes {i+1}: {v:.4f}")

pca = PCA(n_components=0.90)
X_train_pca = pca.fit_transform(x_train)
X_test_pca = pca.transform(x_test)

print("Número de componentes finales:", pca.n_components_)

x_train = pd.DataFrame(
    X_train_pca,
    columns=[f'PC{i+1}' for i in range(X_train_pca.shape[1])],
    index=x_train.index)

x_test = pd.DataFrame(
    X_test_pca,
    columns=[f'PC{i+1}' for i in range(X_test_pca.shape[1])],
    index=x_test.index)

Componentes 1: 0.0900
Componentes 2: 0.1591
Componentes 3: 0.2276
Componentes 4: 0.2952
Componentes 5: 0.3627
Componentes 6: 0.4295
Componentes 7: 0.4961
Componentes 8: 0.5627
Componentes 9: 0.6170
Componentes 10: 0.6708
Componentes 11: 0.7243
Componentes 12: 0.7618
Componentes 13: 0.7918
Componentes 14: 0.8178
Componentes 15: 0.8414
Componentes 16: 0.8645
Componentes 17: 0.8872
Componentes 18: 0.9098
Componentes 19: 0.9323
Componentes 20: 0.9544
Componentes 21: 0.9713
Componentes 22: 0.9880
Componentes 23: 0.9989
Componentes 24: 1.0000
Número de componentes finales: 18
